# Open-source code and data and their impact in the progress of research in Music Information Retrieval

## 1. Retrieving ISMIR Papers with Code from Arxiv 

Not all of the papers from 2011 to 2020 of ISMIR are uploaded to Arxiv, this reminds the importance of all-in-one platform for this purpose to easily collect data for such researches. Here, via web scrapping, we get the papers that are uploaded to Arxiv and retrieve the information on whether they provided their code/data.

In [ ]:
import os, re, time
import json

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.firefox import GeckoDriverManager

import pandas as pd

import urllib, json, re

import os
import json
from collections import Counter

## 1.1. Get corresponding arxiv links for the selected papers

In [428]:
path_to_proceeding_json = "selected_years/"

In [17]:
def arxiv_query(max_result=1):
    paper_urls = {}
    for d,r,f in os.walk(path_to_proceeding_json):
        for file in f:
            if file.endswith(".json"):
                titles = []
                try:
                    with open(path_to_json+file, 'r') as output:
                        f_out = json.load(output)
                        for keys in f_out:
                            
                            #Since the paper titles may include characthers such as .,-,?
                            #They need to be replaced with space
                            #This query method can be improved
                            paper_title = keys['title'].lower().replace(" ","+").replace(".","").replace(",","").replace(":","").replace(";","").replace("?","").replace("-","").replace("'","").replace("-","+")
                            
                            #Create the query for given paper to use Arxiv API
                            url = 'http://export.arxiv.org/api/query?search_query=ti:{}&start=0&max_results={}'.format(paper_title, max_result)
                            
                            with urllib.request.urlopen(url) as ur:
                                r = ur.read()
                            
                            #The links to the arxiv page are stored within this element
                            #A basic regex search
                            match = re.findall('<link title="pdf" href=(.*)',r.decode("utf-8"))
                            for paper_url in match:
                                paper = paper_url.split(" ")[0].split('"')[1]
                                paper_urls[keys['title']] = paper.replace("pdf","abs")
                except:
                    pass
    return paper_urls


In [427]:
#paper_urls = arxiv_query()

In [19]:
paper_urls

{'Four-way Classification of Tabla Strokes with Models Adapted from Automatic Drum Transcription': 'http://arxiv.org/abs/2104.09064v1',
 'A Contextual Latent Space Model: Subsequence Modulation in Melodic Sequence': 'http://arxiv.org/abs/1206.6441v1',
 'OMR-assisted transcription: a case study with early prints': 'http://arxiv.org/abs/1809.05501v1',
 'Deeper Convolutional Neural Networks and Broad Augmentation Policies Improve Performance in Musical Key Estimation': 'http://arxiv.org/abs/2007.09296v1',
 'The Music Performance Markup Format and Ecosystem': 'http://arxiv.org/abs/0712.4219v1',
 'Identification of rhythm guitar sections in symbolic tablatures': 'http://arxiv.org/abs/2107.14653v1',
 'On-Line Audio-to-Lyrics Alignment Based on a Reference Performance': 'http://arxiv.org/abs/1512.02393v2',
 'Visualizing Intertextual Form with Arc Diagrams: Contour and Schema-based Methods': 'http://arxiv.org/abs/0803.4074v2',
 'Unsupervised Domain Adaptation for Document Analysis of Music Sco

In [20]:
#Convert to CSV for file type consistency in upcoming steps
df = pd.DataFrame({'titles':list(paper_urls.keys()),'arxiv_links':list(paper_urls.values())})

In [21]:
#Save the data
df.to_csv("titles_arxiv_links.csv", index=None)

## 1.2. Retrieve Links of Publicly Available Code/Data

In [429]:
df = pd.read_csv("titles_arxiv_links.csv")
papers = df.to_dict()

In [30]:
papers_dict = {}
for title, link in zip(papers['titles'].values(),papers['arxiv_links'].values()):
    papers_dict[title] = link

In [31]:
def get_code_links(urls):
    driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    paper_code = {}
    for title, url in urls.items():
        driver.get(url)
        driver.delete_all_cookies()
        try:
            tit = driver.find_element_by_xpath('//h1[@class="title mathjax"]')
            d = driver.find_element_by_xpath('//div[@id="pwc-output"]/p/a')
            paper_code[title] = {}
            paper_code[title][tit.text] = {url: None}
            if d.get_attribute(name='href'):
                paper_code[title][tit.text][url] = d.get_attribute(name='href')
            else:
                paper_code[title][tit.text][url] = None
        except:
            pass
        main_page = driver.window_handles[0]
        driver.switch_to.window(main_page)
    return paper_code

In [32]:
final_results = get_code_links(papers_dict)



====== WebDriver manager ======
Current firefox version is 94.0
Get LATEST geckodriver version for 94.0 firefox
Driver [/home/seatabay/.wdm/drivers/geckodriver/linux64/v0.30.0/geckodriver] found in cache
/home/seatabay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


Four-way Classification of Tabla Strokes with Models Adapted from Automatic Drum Transcription http://arxiv.org/abs/2104.09064v1


/home/seatabay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # Remove the CWD from sys.path while we load stuff.
/home/seatabay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # This is added back by InteractiveShellApp.init_path()


A Contextual Latent Space Model: Subsequence Modulation in Melodic Sequence http://arxiv.org/abs/1206.6441v1
OMR-assisted transcription: a case study with early prints http://arxiv.org/abs/1809.05501v1
Deeper Convolutional Neural Networks and Broad Augmentation Policies Improve Performance in Musical Key Estimation http://arxiv.org/abs/2007.09296v1
The Music Performance Markup Format and Ecosystem http://arxiv.org/abs/0712.4219v1
Identification of rhythm guitar sections in symbolic tablatures http://arxiv.org/abs/2107.14653v1
On-Line Audio-to-Lyrics Alignment Based on a Reference Performance http://arxiv.org/abs/1512.02393v2
Visualizing Intertextual Form with Arc Diagrams: Contour and Schema-based Methods http://arxiv.org/abs/0803.4074v2
Unsupervised Domain Adaptation for Document Analysis of Music Score Images http://arxiv.org/abs/1804.00347v1
Codified audio language modeling learns useful representations for music information retrieval http://arxiv.org/abs/2107.05677v1
Variable-Lengt

Verifying Music Tag Annotation Via Association Analysis. http://arxiv.org/abs/1907.04262v2
The Role of Audio and Tags in Music Mood Prediction: A Study Using Semantic Layer Projection. http://arxiv.org/abs/1308.1817v1
Converting Path Structures Into Block Structures Using Eigenvalue Decompositions of Self-Similarity Matrices. http://arxiv.org/abs/cond-mat/0104425v1
The Audio Effects Ontology. http://arxiv.org/abs/1610.02333v1
Exploration of Music Emotion Recognition Based on MIDI. http://arxiv.org/abs/2108.01374v1
Rhythmic Pattern Modeling for Beat and Downbeat Tracking in Musical Audio. http://arxiv.org/abs/2111.00704v1
Large-Scale Cover Song Identification Using Chord Profiles. http://arxiv.org/abs/1707.04680v1
Automatically Identifying Vocal Expressions for Music Transcription. http://arxiv.org/abs/2108.02625v1
Hooked: A Game For Discovering What Makes Music Catchy. http://arxiv.org/abs/math/0109023v2
Hierarchical Classification of Carnatic Music Forms. http://arxiv.org/abs/1711.023

...

Transfer Learning for Music Classification and Regression Tasks. http://arxiv.org/abs/1703.09179v4
Drum Transcription via Joint Beat and Drum Modeling Using Convolutional Recurrent Neural Networks. http://arxiv.org/abs/1806.06676v2
A Formalization of Relative Local Tempo Variations in Collections of Performances. http://arxiv.org/abs/1302.0136v1
Sampling Variations of Sequences for Structured Music Generation. http://arxiv.org/abs/1703.00760v1
Quantized Melodic Contours in Indian Art Music Perception: Application to Transcription. http://arxiv.org/abs/1509.04956v1
Modeling the Multiscale Structure of Chord Sequences Using Polytopic Graphs. http://arxiv.org/abs/1808.05335v1
Structured Training for Large-Vocabulary Chord Recognition. http://arxiv.org/abs/1907.02698v1
Modeling and Digitizing Reproducing Piano Rolls. http://arxiv.org/abs/1606.01368v1
Comparing Offertory Melodies of Five Medieval Christian Chant Traditions. http://arxiv.org/abs/1807.11440v1
Counterpoint by Convolution. http

Lyrics-Based Music Genre Classification Using a Hierarchical Attention Network. http://arxiv.org/abs/1707.04678v1
Metrical-Accent Aware Vocal Onset Detection in Polyphonic Audio. http://arxiv.org/abs/1707.06163v1
Mining Labeled Data from Web-Scale Collections for Vocal Activity Detection in Music. http://arxiv.org/abs/1909.01421v1
Multi-Part Pattern Analysis: Combining Structure Analysis and Source Separation to Discover Intra-Part Repeated Sequences. http://arxiv.org/abs/1507.08761v1
One-Step Detection of Background, Staff Lines, and Symbols in Medieval Music Manuscripts with Convolutional Neural Networks. http://arxiv.org/abs/1707.06828v2
Optical Music Recognition with Convolutional Sequence-to-Sequence Models. http://arxiv.org/abs/1707.04877v1
Re-Visiting the Music Segmentation Problem with Crowdsourcing. http://arxiv.org/abs/math/9909103v1
Singing Voice Separation with Deep U-Net Convolutional Networks. http://arxiv.org/abs/2102.11058v1
Sketching Sonata Form Structure in Selected C

The AcousticBrainz Genre Dataset: Multi-Source, Multi-Level, Multi-Label, and Large-Scale http://arxiv.org/abs/1901.03060v2
Data-Driven Song Recognition Estimation Using Collective Memory Dynamics Models http://arxiv.org/abs/1904.13080v1
Towards Interpretable Polyphonic Transcription with Invertible Neural Networks http://arxiv.org/abs/1909.01622v1
Learning to Generate Music With Sentiment http://arxiv.org/abs/2103.06125v1
Backtracking Search Heuristics for Solving the All-partition Array Problem http://arxiv.org/abs/cs/9308101v1
Modeling and Learning Structural Breaks in Sonata Forms http://arxiv.org/abs/1706.09088v2
Auto-adaptive Resonance Equalization using Dilated Residual Networks http://arxiv.org/abs/2106.13227v1
Analyzing User Interactions with Music Information Retrieval System: An Eye-tracking Approach http://arxiv.org/abs/1810.10814v1
A Cross-Scape Plot Representation for Visualizing Symbolic Melodic Similarity http://arxiv.org/abs/2105.05385v1
JosquIntab: A Dataset for Conte

Using Musical Relationships Between Chord Labels in Automatic Chord Extraction Tasks http://arxiv.org/abs/1911.04973v2
A Predictive Model for Music based on Learned Interval Representations http://arxiv.org/abs/1806.08686v1
An End-to-end Framework for Audio-to-Score Music Transcription on Monophonic Excerpts http://arxiv.org/abs/1612.05070v1
Evaluating Automatic Polyphonic Music Transcription http://arxiv.org/abs/2004.07171v1
Onsets and Frames: Dual-Objective Piano Transcription http://arxiv.org/abs/1710.11153v2
Player Vs Transcriber: A Game Approach To Data Manipulation For Automatic Drum Transcription http://arxiv.org/abs/2008.12401v1
Influence in Early Electronic Dance Music: An Audio Content Analysis Investigation. http://arxiv.org/abs/2107.06252v2
Association Mining of Folk Music Genres and Toponyms. http://arxiv.org/abs/1906.12324v1
Characterization of Embellishments in Ney Performances of Makam Music in Turkey. http://arxiv.org/abs/1912.12362v1
Cross-cultural Music Mood Classifi

Fast Identification of Piece and Score Position via Symbolic Fingerprinting. http://arxiv.org/abs/1708.00733v1
A Multimedia Search and Navigation Prototype, Including Music and Video-clips. http://arxiv.org/abs/2008.00781v2
Chord Recognition Using Duration-explicit Hidden Markov Models. http://arxiv.org/abs/2005.07091v2
Finding Repeating Stanzas in Folk Songs. http://arxiv.org/abs/2007.09811v1
Detecting Episodes with Harmonic Sequences for Fugue Analysis. http://arxiv.org/abs/1511.01233v3
Inferring Chord Sequence Meanings via Lyrics: Process and Evaluation. http://arxiv.org/abs/1706.08609v2
Facilitating Comprehensive Benchmarking Experiments on the Million Song Dataset. http://arxiv.org/abs/1712.02289v1
Decoding Tempo and Timing Variations in Music Recordings from Beat Annotations. http://arxiv.org/abs/1710.05608v1
Unsupervised Chord-Sequence Generation from an Audio Example. http://arxiv.org/abs/2110.03209v1
Listening Level Changes Music Similarity. http://arxiv.org/abs/2009.09935v1
P

In [34]:
csv_dic = {}
paper_title = []
arxiv_title = []
arxiv_link = []
code_link = []
for key, val in final_results.items():
    paper_title.append(key)
    for k2,v2 in val.items():
        arxiv_title.append(k2)
        for k3,v3 in v2.items():
            arxiv_link.append(k3)
            code_link.append(v3)
csv_dic['paper_title'] = paper_title
csv_dic['arxiv_title'] = arxiv_title
csv_dic['arxiv_link'] = arxiv_link
csv_dic['code_link'] = code_link
    

In [35]:
df_arxiv_res = pd.DataFrame(csv_dic)

In [36]:
df_arxiv_res.to_csv('arxiv_implementation_results.csv', index=None)

On top of this approach, the papers were manually annotated such as `with code/without code`. 

Since not all of the papers of ISMIR are uploaded to Arxiv and the query may fail in some cases, the output file `arxiv_implementation_results.csv` is manually filtered by comparing the title of the paper and the title of the paper from Arxiv. The merged version is `ismir_2011_2021_with_implementation.csv`.

For the next steps, we should have three different data: CSV with:

1. Titles, year of all papers

2. Titles, year of the papers with code

3. Titles, year of the papers without code


In [470]:
title_year_dic = {}
for d,r,f in os.walk(path_to_json):
    for file in f:
        if file.endswith(".json"):
            with open(path_to_proceeding_json+file, 'r') as output:
                f_out = json.load(output)
            for keys in f_out:
                title_year_dic[keys['title']] = file.split(".")[0]
df_all_papers = pd.DataFrame(title_year_dic, index=[0])
df_all_papers = df_all_papers.T
df_all_papers.reset_index(inplace=True)
df_all_papers.rename(columns={'index':'titles',0:'year'},inplace=True)
df_all_papers.to_csv("ismir_2011_2020_papers.csv", index=None)

In [474]:
df_all_papers.head(3)

,titles,year
0,Multiscale Approaches To Music Audio Feature L...,2013
1,Transfer Learning In Mir: Sharing Learned Late...,2013
2,A Distributed Model For Multiple-Viewpoint Mel...,2013


In [476]:
df_papers_with_code = pd.read_csv("ismir_2011_2020_with_implementation.csv")

In [477]:
df_papers_without_code = df_all_papers[~df_all_papers['titles'].isin(df_papers_with_code['titles'].to_list())]

In [480]:
df_papers_without_code.head(3)

,titles,year
0,Multiscale Approaches To Music Audio Feature L...,2013
1,Transfer Learning In Mir: Sharing Learned Late...,2013
2,A Distributed Model For Multiple-Viewpoint Mel...,2013


In [481]:
df_papers_without_code.to_csv("ismir_2011_2020_without_implementation.csv")

## 2. **Creating subcategories of the papers**

ISMIR Papers do not have a keywords section & this leads to a problem on the categorisation of the papers. Since there isn't a common agreement on the subtopics of Music Information Retrieval, for this research, we decided to use `http://www.music.mcgill.ca/~ich/classes/mumt621_15/MIR_topics.html` as base and extended the keywords by iterating over the titles of the papers. This is not a solid solution but such a data is required for further analysis on open data/code in MIR.

This subtopics are stored in JSON format: `mir_topics.json`

In [ ]:
with open('mir_topics.json', 'r') as mir_out:
    mir_topics = json.load(mir_out)

In [482]:
def categorise_papers(df):
    df['category'] = None
    for idx, row in df.iterrows():
        row = row.copy()
        categories = []
        for key, val in mir_topics.items():
            for v in val:
                title = [t.lower() for t in row.titles.replace("-"," ").replace(":"," ").split(" ")]
                if v.lower() in title:
                    categories.append(key)
        if categories == []:
            categories = "Other"
        else:
            categories = ",".join(cat for cat in set(categories))
        df.loc[idx, 'category']= categories
    return df

In [500]:
df = categorise_papers(df_all_papers)
df_papers_with_code_cat = categorise_papers(df_papers_with_code)
df_papers_without_code_cat = categories(df_papers_without_code)

TypeError: 'str' object is not callable

In [484]:
df.head(5)

,titles,year,category
0,Multiscale Approaches To Music Audio Feature L...,2013,Other
1,Transfer Learning In Mir: Sharing Learned Late...,2013,Knowledge representation
2,A Distributed Model For Multiple-Viewpoint Mel...,2013,Other
3,Learning Rhythm And Melody Features With Deep ...,2013,"Transcription and annotations,Recognition / An..."
4,A Comparative Study Of Indian And Western Musi...,2013,Other


In [501]:
df.to_csv("ismir_2011_2020_papers_categorized.csv", index=None)
df_papers_with_code_cat.to_csv("ismir_2011_2020_papers_with_code_categorized.csv", index=None)
df_papers_without_code.to_csv("ismir_2011_2020_papers_without_code_categorized.csv", index=None)

## 3. Finding Papers Published an Open Dataset

As an intuition, we could say that dataset/database creation among MIR community has increased but this can be analyzed by searching for the amount of published papers for dataset/database creation. To do that, our approach is to search for the titles including words such as `dataset` and `database`.

The next step is to manually check the results.

In [494]:
dataset = {}
for idx, row in df_all_papers.iterrows():
    for single_word in row.titles.replace(".","").split(" "):
        if single_word.lower() == 'dataset':
            dataset[row.titles] = row.year

In [495]:
dataset

{'The Million Musical Tweet Dataset - What We Can Learn From Microblogs.': '2013',
 'QBT-Extended: An Annotated Dataset of Melodically Contoured Tapped Queries.': '2013',
 'MedleyDB: A Multitrack Dataset for Annotation-Intensive MIR Research.': '2014',
 'Introducing a Dataset of Emotional and Color Responses to Music.': '2014',
 'Automatic Figured Bass Annotation Using the New Bach Chorales Figured Bass Dataset': '2020',
 'POP909: A Pop-Song Dataset for Music Arrangement Generation': '2020',
 'ASAP: a dataset of aligned scores and performances for piano transcription': '2020',
 'The Freesound Loop Dataset and Annotation Tool': '2020',
 'dMelodies: A Music Dataset for Disentanglement Learning': '2020',
 'The Million Song Dataset.': '2011',
 'Automatic Practice Logging: Introduction, Dataset & Preliminary Study.': '2016',
 'The Music Listening Histories Dataset.': '2017',
 'FMA: A Dataset for Music Analysis.': '2017',
 'NMED-T: A Tempo-Focused Dataset of Cortical and Behavioral Responses

In [499]:
df_dataset = pd.DataFrame(dataset, index=[0])
df_dataset = df_dataset.T
df_dataset.reset_index(inplace=True)
df_dataset.rename(columns={'index':'titles',0:'year'},inplace=True)
df_dataset.to_csv("ismir_2011_2020_dataset.csv", index=None)